In [2]:
import os
import findspark
findspark.init()

from pyspark.sql import SparkSession
os.environ['SPARK_HOME']='/Users/audioworkstation/Documents/WORKSPACE/LEARNING/pyspark-training/spark'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='lab'
os.environ['PYSPARK_PYTHON']='python'


spark = SparkSession.builder.appName("Dataframe Operations").getOrCreate()

23/11/10 11:31:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
%%bash 
head -n 10 ./resources/stocks.txt

id,name,category,quantity,price
1,iPhone,Electronics,10,899.99
2,Macbook,Electronics,5,1299.99
3,iPad,Electronics,15,499.99
4,Samsung TV,Electronics,8,799.99
5,LG TV,Electronics,10,699.99
6,Nike Shoes,Clothing,30,99.99
7,Adidas Shoes,Clothing,25,89.99
8,Sony Headphones,Electronics,12,149.99
9,Beats Headphones,Electronics,20,199.99


In [7]:
df = spark.read.csv(
    './resources/stocks.txt',
    header=True,
    inferSchema=True
)
df.show(10)

+---+----------------+-----------+--------+-------+
| id|            name|   category|quantity|  price|
+---+----------------+-----------+--------+-------+
|  1|          iPhone|Electronics|      10| 899.99|
|  2|         Macbook|Electronics|       5|1299.99|
|  3|            iPad|Electronics|      15| 499.99|
|  4|      Samsung TV|Electronics|       8| 799.99|
|  5|           LG TV|Electronics|      10| 699.99|
|  6|      Nike Shoes|   Clothing|      30|  99.99|
|  7|    Adidas Shoes|   Clothing|      25|  89.99|
|  8| Sony Headphones|Electronics|      12| 149.99|
|  9|Beats Headphones|Electronics|      20| 199.99|
| 10|    Dining Table|  Furniture|      10| 249.99|
+---+----------------+-----------+--------+-------+
only showing top 10 rows



### SELECT

In [12]:
df.select('id', 'name', 'category').show()

+---+----------------+-----------+
| id|            name|   category|
+---+----------------+-----------+
|  1|          iPhone|Electronics|
|  2|         Macbook|Electronics|
|  3|            iPad|Electronics|
|  4|      Samsung TV|Electronics|
|  5|           LG TV|Electronics|
|  6|      Nike Shoes|   Clothing|
|  7|    Adidas Shoes|   Clothing|
|  8| Sony Headphones|Electronics|
|  9|Beats Headphones|Electronics|
| 10|    Dining Table|  Furniture|
| 11|      Study Desk|  Furniture|
| 12|          Apples|       Food|
| 13|         Bananas|       Food|
| 14|         Oranges|       Food|
| 15|  Chicken Breast|       Food|
| 16|   Salmon Fillet|       Food|
| 17|  Leather Jacket|   Clothing|
| 18|     Winter Coat|   Clothing|
| 19|        Yoga Mat|     Sports|
| 20|    Dumbbell Set|     Sports|
+---+----------------+-----------+
only showing top 20 rows



### FILTER

In [13]:
df.filter(df.quantity > 10).show()

+---+----------------+-----------+--------+------+
| id|            name|   category|quantity| price|
+---+----------------+-----------+--------+------+
|  3|            iPad|Electronics|      15|499.99|
|  6|      Nike Shoes|   Clothing|      30| 99.99|
|  7|    Adidas Shoes|   Clothing|      25| 89.99|
|  8| Sony Headphones|Electronics|      12|149.99|
|  9|Beats Headphones|Electronics|      20|199.99|
| 12|          Apples|       Food|     100|   0.5|
| 13|         Bananas|       Food|     150|  0.25|
| 14|         Oranges|       Food|     120|  0.75|
| 15|  Chicken Breast|       Food|      50|  3.99|
| 16|   Salmon Fillet|       Food|      30|  5.99|
| 17|  Leather Jacket|   Clothing|      15|199.99|
| 19|        Yoga Mat|     Sports|      20| 19.99|
| 20|    Dumbbell Set|     Sports|      15| 49.99|
| 24|      Laptop Bag|Accessories|      25| 29.99|
| 25|        Backpack|Accessories|      30| 24.99|
| 28|           Jeans|   Clothing|      30| 59.99|
| 29|         T-shirt|   Clothi

### GroupBy

In [15]:
df.groupby('category').count().show()

+-----------+-----+
|   category|count|
+-----------+-----+
|       Food|    5|
|     Sports|    2|
|Electronics|    9|
|   Clothing|    7|
|  Furniture|    5|
|Accessories|    2|
+-----------+-----+



In [22]:
df.groupby('category').agg({
    "quantity":"sum",
    "price":"avg",
    "id":"count"
}).show()

+-----------+-------------+------------------+---------+
|   category|sum(quantity)|        avg(price)|count(id)|
+-----------+-------------+------------------+---------+
|       Food|          450|2.2960000000000003|        5|
|     Sports|           35|             34.99|        2|
|Electronics|           98| 586.6566666666665|        9|
|   Clothing|          200|  99.2757142857143|        7|
|  Furniture|           41|            141.99|        5|
|Accessories|           55|             27.49|        2|
+-----------+-------------+------------------+---------+



### JOINS

In [24]:
df_2 = df.select('id', 'category').limit(5)
df.join(df_2, 'id', 'inner').show()

+---+----------+-----------+--------+-------+-----------+
| id|      name|   category|quantity|  price|   category|
+---+----------+-----------+--------+-------+-----------+
|  1|    iPhone|Electronics|      10| 899.99|Electronics|
|  2|   Macbook|Electronics|       5|1299.99|Electronics|
|  3|      iPad|Electronics|      15| 499.99|Electronics|
|  4|Samsung TV|Electronics|       8| 799.99|Electronics|
|  5|     LG TV|Electronics|      10| 699.99|Electronics|
+---+----------+-----------+--------+-------+-----------+



### SORT

In [32]:
df.orderBy('price').show()

+---+---------------+-----------+--------+------+
| id|           name|   category|quantity| price|
+---+---------------+-----------+--------+------+
| 13|        Bananas|       Food|     150|  0.25|
| 12|         Apples|       Food|     100|   0.5|
| 14|        Oranges|       Food|     120|  0.75|
| 15| Chicken Breast|       Food|      50|  3.99|
| 16|  Salmon Fillet|       Food|      30|  5.99|
| 29|        T-shirt|   Clothing|      50| 14.99|
| 19|       Yoga Mat|     Sports|      20| 19.99|
| 25|       Backpack|Accessories|      30| 24.99|
| 24|     Laptop Bag|Accessories|      25| 29.99|
| 20|   Dumbbell Set|     Sports|      15| 49.99|
| 28|          Jeans|   Clothing|      30| 59.99|
| 22|   Office Chair|  Furniture|       8| 79.99|
| 30|       Sneakers|   Clothing|      40| 79.99|
|  7|   Adidas Shoes|   Clothing|      25| 89.99|
|  6|     Nike Shoes|   Clothing|      30| 99.99|
| 23|      Bookshelf|  Furniture|      10| 99.99|
| 21|   Coffee Table|  Furniture|       5|129.99|


In [33]:
from pyspark.sql.functions import desc, col

df.orderBy(col('price').desc(), col('quantity').desc()).show()


+---+----------------+-----------+--------+-------+
| id|            name|   category|quantity|  price|
+---+----------------+-----------+--------+-------+
|  2|         Macbook|Electronics|       5|1299.99|
|  1|          iPhone|Electronics|      10| 899.99|
|  4|      Samsung TV|Electronics|       8| 799.99|
|  5|           LG TV|Electronics|      10| 699.99|
| 26|          Camera|Electronics|      10| 599.99|
|  3|            iPad|Electronics|      15| 499.99|
| 10|    Dining Table|  Furniture|      10| 249.99|
|  9|Beats Headphones|Electronics|      20| 199.99|
| 17|  Leather Jacket|   Clothing|      15| 199.99|
|  8| Sony Headphones|Electronics|      12| 149.99|
| 18|     Winter Coat|   Clothing|      10| 149.99|
| 11|      Study Desk|  Furniture|       8| 149.99|
| 27|         Printer|Electronics|       8| 129.99|
| 21|    Coffee Table|  Furniture|       5| 129.99|
|  6|      Nike Shoes|   Clothing|      30|  99.99|
| 23|       Bookshelf|  Furniture|      10|  99.99|
|  7|    Adi

### DISTINCT

In [35]:
df.select('category').distinct().show()

+-----------+
|   category|
+-----------+
|       Food|
|     Sports|
|Electronics|
|   Clothing|
|  Furniture|
|Accessories|
+-----------+



### DROP

In [36]:
df.drop('quantity').show()

+---+----------------+-----------+-------+
| id|            name|   category|  price|
+---+----------------+-----------+-------+
|  1|          iPhone|Electronics| 899.99|
|  2|         Macbook|Electronics|1299.99|
|  3|            iPad|Electronics| 499.99|
|  4|      Samsung TV|Electronics| 799.99|
|  5|           LG TV|Electronics| 699.99|
|  6|      Nike Shoes|   Clothing|  99.99|
|  7|    Adidas Shoes|   Clothing|  89.99|
|  8| Sony Headphones|Electronics| 149.99|
|  9|Beats Headphones|Electronics| 199.99|
| 10|    Dining Table|  Furniture| 249.99|
| 11|      Study Desk|  Furniture| 149.99|
| 12|          Apples|       Food|    0.5|
| 13|         Bananas|       Food|   0.25|
| 14|         Oranges|       Food|   0.75|
| 15|  Chicken Breast|       Food|   3.99|
| 16|   Salmon Fillet|       Food|   5.99|
| 17|  Leather Jacket|   Clothing| 199.99|
| 18|     Winter Coat|   Clothing| 149.99|
| 19|        Yoga Mat|     Sports|  19.99|
| 20|    Dumbbell Set|     Sports|  49.99|
+---+------

### Add new column

In [38]:
df.withColumn('total_price', df.quantity * df.price).show()

+---+----------------+-----------+--------+-------+------------------+
| id|            name|   category|quantity|  price|       total_price|
+---+----------------+-----------+--------+-------+------------------+
|  1|          iPhone|Electronics|      10| 899.99|            8999.9|
|  2|         Macbook|Electronics|       5|1299.99|           6499.95|
|  3|            iPad|Electronics|      15| 499.99|           7499.85|
|  4|      Samsung TV|Electronics|       8| 799.99|           6399.92|
|  5|           LG TV|Electronics|      10| 699.99|            6999.9|
|  6|      Nike Shoes|   Clothing|      30|  99.99|            2999.7|
|  7|    Adidas Shoes|   Clothing|      25|  89.99|           2249.75|
|  8| Sony Headphones|Electronics|      12| 149.99|           1799.88|
|  9|Beats Headphones|Electronics|      20| 199.99|            3999.8|
| 10|    Dining Table|  Furniture|      10| 249.99|            2499.9|
| 11|      Study Desk|  Furniture|       8| 149.99|           1199.92|
| 12| 

### Alias: rename column for readability 

In [40]:
df.withColumnRenamed('price', 'product_price').show()

+---+----------------+-----------+--------+-------------+
| id|            name|   category|quantity|product_price|
+---+----------------+-----------+--------+-------------+
|  1|          iPhone|Electronics|      10|       899.99|
|  2|         Macbook|Electronics|       5|      1299.99|
|  3|            iPad|Electronics|      15|       499.99|
|  4|      Samsung TV|Electronics|       8|       799.99|
|  5|           LG TV|Electronics|      10|       699.99|
|  6|      Nike Shoes|   Clothing|      30|        99.99|
|  7|    Adidas Shoes|   Clothing|      25|        89.99|
|  8| Sony Headphones|Electronics|      12|       149.99|
|  9|Beats Headphones|Electronics|      20|       199.99|
| 10|    Dining Table|  Furniture|      10|       249.99|
| 11|      Study Desk|  Furniture|       8|       149.99|
| 12|          Apples|       Food|     100|          0.5|
| 13|         Bananas|       Food|     150|         0.25|
| 14|         Oranges|       Food|     120|         0.75|
| 15|  Chicken

In [41]:
spark.stop()